# Exercício Prático de Análise de Coorte em Python

Este notebook irá guiá-lo através de um exercício simples para criar uma tabela de coorte usando Python e a biblioteca Pandas. O objetivo é entender como agrupar usuários por sua primeira ação (neste caso, a primeira compra) e analisar seu comportamento ao longo do tempo.

**Passos que iremos seguir:**
1. **Criar Dados de Exemplo:** Vamos gerar um DataFrame simples com dados de pedidos.
2. **Identificar a Data da Primeira Compra:** Para cada cliente, encontraremos a data do seu primeiro pedido.
3. **Criar Coortes Mensais:** Agrupar os clientes pelo mês da sua primeira compra.
4. **Analisar o Comportamento da Coorte:** Contar o número de clientes ativos em cada mês para cada coorte.
5. **Criar a Tabela de Coorte:** Usar uma tabela dinâmica para visualizar a retenção de clientes por coorte.

## 1. Criar Dados de Exemplo

Vamos criar um DataFrame Pandas simulando dados de pedidos online. Este DataFrame terá as seguintes colunas:
- `order_date`: Data do pedido.
- `customer_id`: ID do cliente.
- `revenue`: Valor do pedido.

In [1]:
import pandas as pd

# Criando um dicionário com dados de exemplo
data = {
    'order_date': pd.to_datetime(['2023-01-15', '2023-01-20', '2023-02-05', '2023-02-10', '2023-01-25', '2023-02-15', '2023-03-01', '2023-03-10', '2023-03-20', '2023-03-25'] * 3),
    'customer_id': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5] * 3,
    'revenue': [50, 60, 70, 80, 90, 100, 110, 120, 130, 140] * 3
}

# Criando o DataFrame
orders = pd.DataFrame(data)

print(orders.head())

  order_date  customer_id  revenue
0 2023-01-15            1       50
1 2023-01-20            1       60
2 2023-02-05            2       70
3 2023-02-10            2       80
4 2023-01-25            3       90


## 2. Identificar a Data da Primeira Compra

Agora, vamos encontrar a data do primeiro pedido para cada cliente. Usaremos `groupby()` por `customer_id` e `.min()` na coluna `order_date`.

In [2]:
# Encontrando a data do primeiro pedido para cada cliente
first_order_date_by_customers = orders.groupby('customer_id')['order_date'].min()

# Renomeando a Series para facilitar o merge
first_order_date_by_customers.name = 'first_order_date'

print(first_order_date_by_customers)

customer_id
1   2023-01-15
2   2023-02-05
3   2023-01-25
4   2023-03-01
5   2023-03-20
Name: first_order_date, dtype: datetime64[ns]


## 3. Criar Coortes Mensais

Vamos juntar a data do primeiro pedido de volta ao DataFrame `orders` e criar colunas para o mês do pedido e o mês da primeira compra para definir nossas coortes.

In [3]:
# Juntando a data do primeiro pedido ao DataFrame original
orders = orders.join(first_order_date_by_customers, on='customer_id')

# Criando colunas para o mês do pedido e o mês da primeira compra
orders['order_month'] = orders['order_date'].dt.to_period('M')
orders['first_order_month'] = orders['first_order_date'].dt.to_period('M')

print(orders.head())

  order_date  customer_id  revenue first_order_date order_month  \
0 2023-01-15            1       50       2023-01-15     2023-01   
1 2023-01-20            1       60       2023-01-15     2023-01   
2 2023-02-05            2       70       2023-02-05     2023-02   
3 2023-02-10            2       80       2023-02-05     2023-02   
4 2023-01-25            3       90       2023-01-25     2023-01   

  first_order_month  
0           2023-01  
1           2023-01  
2           2023-02  
3           2023-02  
4           2023-01  


## 4. Analisar o Comportamento da Coorte

Agora, vamos agrupar os dados por coorte (mês da primeira compra) e mês do pedido para contar o número de clientes únicos em cada grupo.

In [4]:
# Agrupando por coorte e mês do pedido e contando clientes únicos
cohort_analysis = orders.groupby(['first_order_month', 'order_month']).agg({'customer_id': 'nunique'}).reset_index()

print(cohort_analysis)

  first_order_month order_month  customer_id
0           2023-01     2023-01            2
1           2023-01     2023-02            1
2           2023-02     2023-02            1
3           2023-03     2023-03            2


## 5. Criar a Tabela de Coorte

Finalmente, vamos usar `pivot_table` para formatar nossos resultados em uma tabela de coorte, onde as linhas representam as coortes (mês da primeira compra) e as colunas representam os meses subsequentes.

In [5]:
# Criando a tabela dinâmica de coorte
cohort_pivot = cohort_analysis.pivot_table(index='first_order_month', columns='order_month', values='customer_id')

print(cohort_pivot)

order_month        2023-01  2023-02  2023-03
first_order_month                           
2023-01                2.0      1.0      NaN
2023-02                NaN      1.0      NaN
2023-03                NaN      NaN      2.0


## Exercício Adicional

**Tarefa:** Modifique o código para calcular a receita total por coorte e mês do pedido, em vez de contar o número de clientes únicos. Crie uma nova tabela dinâmica que mostre a receita total por coorte ao longo do tempo.

**Dica:** Em vez de usar `'customer_id': 'nunique'` na função `agg()`, use `'revenue': 'sum'`. Ajuste também a função `pivot_table` para usar os valores de receita.